# Cross validation

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

x = digits.data
y = digits.target

## 测试train_test_split

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state = 666)

In [4]:
# 使用超参数的方法进行调整

from sklearn.neighbors import KNeighborsClassifier

best_score, best_k, best_p = 0, 0, 0

for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p, n_jobs=-1)
        knn_clf.fit(x_train, y_train)
        score = knn_clf.score(x_test, y_test)
        
        if score > best_score:
            best_score, best_k, best_p = score, k, p

print("Best K = ", best_k)
print("Best P = ", best_p)
print("Best score = ", best_score)        

Best K =  3
Best P =  4
Best score =  0.9860917941585535


## 使用交叉验证的方法进行调整

In [5]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier(n_jobs=-1)
cross_val_score(knn_clf, x_train, y_train, cv=3)

array([0.98895028, 0.97777778, 0.96629213])

In [6]:
best_score, best_k, best_p = 0, 0, 0

for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p, n_jobs=-1)
        scores = cross_val_score(knn_clf, x_train, y_train, cv=3)
        score = np.mean(scores)
        
        if score > best_score:
            best_score, best_k, best_p = score, k, p

print("Best K = ", best_k)
print("Best P = ", best_p)
print("Best score = ", best_score)   

Best K =  2
Best P =  2
Best score =  0.9823599874006478


In [7]:
best_knn = KNeighborsClassifier(n_neighbors=2, p=2, n_jobs=-1)
best_knn.fit(x_train, y_train)
score = best_knn.score(x_test, y_test)

In [8]:
score

0.980528511821975

## 回顾网格搜索方法(内部包含交叉验证)

In [9]:
from sklearn.model_selection import GridSearchCV

# 参数设置
param_grid = [{
    "weights":["distance"],
    "n_neighbors":[i for i in range(2, 11)],
    "p":[i for i in range(1, 6)]
}]

knn = KNeighborsClassifier(n_jobs=-1)
grid_search = GridSearchCV(knn, param_grid, verbose=2, cv = 3, n_jobs=-1)
%time grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.7s


Wall time: 7.52 s


[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    7.3s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [10]:
grid_search.best_score_

0.9823747680890538

In [11]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [12]:
bes_knn_clf = grid_search.best_estimator_
bes_knn_clf.score(x_test, y_test)

0.980528511821975

# 总结：

* 1、交叉验证主要是用来选择最好的参数，即选择超参数
* 2、网格搜索方法中使用的正是交叉验证的方法